In [ ]:
import numpy as np
import os
from sklearn.decomposition import PCA, TruncatedSVD
import cv2
from scipy.signal import convolve2d
from sklearn.impute import KNNImputer
from scipy.ndimage import median_filter
from tqdm import tqdm


In [ ]:
def preprocess_images(batch_size,base_dir, target_size=(64, 64)):
    processed_images = []
    img_dir = os.listdir(base_dir)
    
    for i in tqdm(range(0, len(img_dir), batch_size)):
        batch_files = img_dir[i:i + batch_size]
        batch_images = []
        for filename in batch_files:
            filepath = os.path.join(base_dir, filename)
            img = cv2.imread(filepath) # read image
            img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA) # resize image
            batch_images.append(img)
        
        if not batch_images:
            # nothing to process in this batch
            continue
        processed_images.extend(batch_images)
    return np.array(processed_images)


def reduce_dimensions(images, reduction_method='pca', variance_threshold=0.95):
    """Apply dimension reduction using PCA or SVD"""
    original_shape = images.shape
    flattened = images.reshape(len(images), -1)
    
    if reduction_method.lower() == 'pca':
        reducer = PCA(n_components=variance_threshold)
    else:
        reducer = TruncatedSVD(n_components=min(flattened.shape))
    
    reduced = reducer.fit_transform(flattened)
    reconstructed = reducer.inverse_transform(reduced)
    variance_preserved = sum(reducer.explained_variance_ratio_) * 100
    
    print(f"Variance preserved: {variance_preserved:.2f}%")
    return reconstructed.reshape(original_shape), variance_preserved

def apply_convolution(image, kernel):
    """Apply convolution to an image"""
    if len(image.shape) == 3:
        return np.stack([convolve2d(image[:,:,c], kernel, mode='same', boundary='wrap')
                        for c in range(image.shape[2])], axis=2)
    return convolve2d(image, kernel, mode='same', boundary='wrap')


def get_convolution_kernels():
    """Return dictionary of common convolution kernels"""
    return {
        'edge_detection': np.array([[-1, -1, -1],
                                  [-1,  8, -1],
                                  [-1, -1, -1]]),
        'sobel_x': np.array([[-1, 0, 1],
                            [-2, 0, 2],
                            [-1, 0, 1]]),
        'sobel_y': np.array([[-1, -2, -1],
                            [ 0,  0,  0],
                            [ 1,  2,  1]]),
        'color_contrast': np.array([[0, -1, 0],
                                  [-1, 5, -1],
                                  [0, -1, 0]])
    }

def apply_max_pooling(image, pool_size=2):
    """Apply max pooling to an image"""
    h, w = image.shape[:2]
    new_h, new_w = h//pool_size, w//pool_size
    pooled = np.zeros((new_h, new_w) + image.shape[2:])
    
    for i in range(new_h):
        for j in range(new_w):
            pooled[i,j] = np.max(image[i*pool_size:(i+1)*pool_size, 
                                     j*pool_size:(j+1)*pool_size], axis=(0,1))
    return pooled

def apply_avg_pooling(image, pool_size=2):
    """Apply average pooling to an image"""
    h, w = image.shape[:2]
    new_h, new_w = h//pool_size, w//pool_size
    pooled = np.zeros((new_h, new_w) + image.shape[2:])
    
    for i in range(new_h):
        for j in range(new_w):
            pooled[i,j] = np.mean(image[i*pool_size:(i+1)*pool_size, 
                                      j*pool_size:(j+1)*pool_size], axis=(0,1))
    return pooled

def flatten_batch(images):
    """Flatten batch of images"""
    return images.reshape(len(images), -1)

def remove_outliers(image, method='knn', n_neighbors=5, window_size=3):
    """Remove outliers from image using specified method"""
    
    original_shape = image.shape
    h, w = image.shape[:2]
    
    # Handle different color channels separately
    if len(original_shape) == 3:
        cleaned_image = np.zeros_like(image)
        for c in range(original_shape[2]):
            cleaned_image[:,:,c] = remove_outliers(image[:,:,c], method, n_neighbors, window_size)
        return cleaned_image
    
    if method.lower() == 'knn':
        # Prepare data for KNN imputation
        X = np.zeros((h*w, window_size**2))
        for i in range(h):
            for j in range(w):
                # Extract local window
                i_start = max(0, i - window_size//2)
                i_end = min(h, i + window_size//2 + 1)
                j_start = max(0, j - window_size//2)
                j_end = min(w, j + window_size//2 + 1)
                window = image[i_start:i_end, j_start:j_end].flatten()
                # Pad if necessary
                if len(window) < window_size**2:
                    window = np.pad(window, (0, window_size**2 - len(window)), mode='edge')
                X[i*w + j] = window
        
        # Apply KNN imputation
        imputer = KNNImputer(n_neighbors=n_neighbors)
        cleaned = imputer.fit_transform(X)
        
        # Reshape back to image
        return cleaned[:,window_size**2//2].reshape(h, w)
    
    elif method.lower() == 'median':
        return median_filter(image, size=window_size)
    
    else:
        raise ValueError(f"Unknown outlier removal method: {method}")


In [ ]:
A = os.listdir('E:/@IIT_BBS/@Sem 1/ML/Final Project/P1')
len(A)

In [ ]:
demo = cv2.imread('E:/@IIT_BBS/@Sem 1/ML/Final Project/P1/' + A[0])

In [ ]:
demo.shape

In [ ]:
cv2.imwrite('demo.png',demo)


In [ ]:
batch_size = 10  # Adjust this number based on your available memory
# Use the same absolute folder that was used to create A
base_dir = r'E:/@IIT_BBS/@Sem 1/ML/Final Project/P1'



In [ ]:

# ----------------------------------------------------------
# 1. Convolution filter bank
# ----------------------------------------------------------
def apply_conv(image, kernel_size=3):
    """Apply Sobel, Laplacian, Gabor, DoG filters and return multi-channel output."""
    
    # Ensure grayscale
    if image is None:
        raise ValueError("Image loaded as None.")

    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()

    filters = []

    # Sobel X, Sobel Y
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=kernel_size)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=kernel_size)
    filters.extend([sobelx, sobely])

    # Laplacian
    lap = cv2.Laplacian(gray, cv2.CV_64F)
    filters.append(lap)

    # 6 Gabor filters
    for theta in np.arange(0, np.pi, np.pi / 6):
        kernel = cv2.getGaborKernel((11, 11), 4.0, theta, 10.0, 0.5, 0, ktype=cv2.CV_32F)
        fimg = cv2.filter2D(gray, cv2.CV_64F, kernel)
        filters.append(fimg)

    # Difference of Gaussians
    g1 = cv2.GaussianBlur(gray, (3, 3), 1)
    g2 = cv2.GaussianBlur(gray, (5, 5), 2)
    dog = g1 - g2
    filters.append(dog)

    # Stack channels → (H, W, C)
    stacked = np.stack(filters, axis=-1)

    # Normalize per channel (0-1)
    norm = np.zeros_like(stacked, dtype=np.float32)
    for c in range(stacked.shape[-1]):
        ch = stacked[..., c]
        norm[..., c] = (ch - ch.min()) / (ch.max() - ch.min() + 1e-6)

    return norm


# ----------------------------------------------------------
# 2. Deterministic pooling (always ends at target_size × target_size)
# ----------------------------------------------------------
def apply_maxpooling(image, target_pool_size=8):
    """Resize down progressively, then enforce exact output size."""
    
    pooled = image.astype(np.float32)

    def safe_resize(img, new_w, new_h):
        if img.ndim == 2 or img.shape[2] <= 4:
            return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        else:
            # Resize per-channel if many channels
            resized = [cv2.resize(img[..., c], (new_w, new_h), interpolation=cv2.INTER_AREA)
                       for c in range(img.shape[2])]
            return np.stack(resized, axis=-1)

    # Progressive downsampling
    while True:
        h, w = pooled.shape[:2]
        if min(h, w) <= target_pool_size:
            break
        pooled = safe_resize(pooled, max(1, w // 2), max(1, h // 2))

    # Final deterministic size — essential fix
    pooled = safe_resize(pooled, target_pool_size, target_pool_size)
    return pooled


# ----------------------------------------------------------
# 3. Complete feature extractor
# ----------------------------------------------------------
def feature_extraction(image, kernel_size=3, target_pool_size=8):
    """Returns fixed-length handcrafted feature vector."""
    
    conv = apply_conv(image, kernel_size=kernel_size)
    pooled = apply_maxpooling(conv, target_pool_size=target_pool_size)

    # Flatten pooled features
    flat = pooled.flatten()

    # Summary statistics
    means = pooled.mean(axis=(0, 1))
    stds = pooled.std(axis=(0, 1))

    # Final vector is deterministic
    vec = np.concatenate([flat, means, stds])
    return vec



In [ ]:

# ----------------------------------------------------------
# 4. Batch feature extraction
# ----------------------------------------------------------
def batch_CFE(batch_size, base_dir, target_size=8):
    """Batch handcrafted feature extraction."""
    
    processed = []
    img_files = os.listdir(base_dir)

    for i in tqdm(range(0, len(img_files), batch_size)):
        batch_names = img_files[i:i + batch_size]
        
        for fname in batch_names:
            path = os.path.join(base_dir, fname)
            img = cv2.imread(path)

            if img is None:
                print("Skipping unreadable file:", fname)
                continue

            feats = feature_extraction(img, kernel_size=3, target_pool_size=target_size)
            processed.append(feats)

    return np.array(processed)

# feats = feature_extraction(processed_images[9], kernel_size=3,target_pool_size=8)
# print(feats.shape)


In [ ]:
batch_features = batch_CFE(batch_size=20,base_dir=base_dir, target_size=8)
batch_features.shape